In [ ]:
import cv2
import numpy as np

# Function to detect lanes and calculate center offset
def detect_lanes_and_calculate_offset(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Define region of interest (ROI)
    height, width = frame.shape[:2]
    roi_vertices = np.array([[(0, height), (width / 2, height / 2), (width, height)]], dtype=np.int32)
    roi_mask = np.zeros_like(edges)
    cv2.fillPoly(roi_mask, roi_vertices, 255)
    roi_edges = cv2.bitwise_and(edges, roi_mask)
    
    # Detect lines using Hough transform
    lines = cv2.HoughLinesP(roi_edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=100)
    
    # Fit a polynomial to each lane line
    left_lane_pts = []
    right_lane_pts = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        slope = (y2 - y1) / (x2 - x1)
        if slope < 0:  # Left lane
            left_lane_pts.extend([(x1, y1), (x2, y2)])
        elif slope > 0:  # Right lane
            right_lane_pts.extend([(x1, y1), (x2, y2)])
    
    if left_lane_pts and right_lane_pts:
        left_lane_pts = np.array(left_lane_pts)
        right_lane_pts = np.array(right_lane_pts)
        
        left_lane_params = np.polyfit(left_lane_pts[:, 0], left_lane_pts[:, 1], 1)
        right_lane_params = np.polyfit(right_lane_pts[:, 0], right_lane_pts[:, 1], 1)
        
        left_lane_bottom = np.polyval(left_lane_params, height)
        right_lane_bottom = np.polyval(right_lane_params, height)
        
        center_offset = (width / 2) - ((left_lane_bottom + right_lane_bottom) / 2)
    else:
        center_offset = 0
    
    return center_offset

# Function to map center offset to steering angle
def map_offset_to_steering_angle(center_offset):
    # Mapping logic: You may need to adjust this based on your specific setup
    steering_angle = np.arctan(center_offset / 320) * (180 / np.pi)  # Assuming frame width is 640
    
    return steering_angle



In [ ]:
# Main loop
cap = cv2.VideoCapture(0) 
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break
    
    # Detect lanes and calculate center offset
    center_offset = detect_lanes_and_calculate_offset(frame)
    
    # Map center offset to steering angle
    steering_angle = map_offset_to_steering_angle(center_offset)
    
    # Print steering angle (for demonstration)
    print("Steering angle:", steering_angle)
    
    # Display frame with lane detection (for visualization)
    cv2.imshow('Frame', frame)
    
    # Check for key press to quit
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
